# MAIN WORKING BODY

#### Import stuff and creating drivers

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException

from bs4 import BeautifulSoup

import pandas as pd

driver = webdriver.Chrome('/home/jaspreet/Documents/chromedriver')

#### Initialising the Driver with Indeed.co

In [13]:
driver.get('https://www.indeed.co.in/')
driver.refresh()

#### Enter Jobs and location for the first time 

In [7]:
jobs = input('Enter job: ')
loc = input('Enter location: ')

Enter job: Sales
Enter location: Delhu


#### Input the Job and Location into the text fields and search 

In [16]:
what = driver.find_element_by_id('text-input-what')
where = driver.find_element_by_id('text-input-where')

what.send_keys(Keys.CONTROL + "a")
what.send_keys(Keys.DELETE)
what.send_keys(jobs)

where.send_keys(Keys.CONTROL + "a")
where.send_keys(Keys.DELETE)
where.send_keys(loc)

first_suggestion = driver.find_elements_by_xpath("//li[@class = 'icl-Autocomplete-option']")[0]
first_suggestion.click()

where.send_keys(Keys.RETURN)

#### Place holding DF till I figure out something better.

In [57]:
df = pd.DataFrame(columns=('Job_Title', 'Company_Name', 'Location', 'Salary', 'Job_Summary'))

#### Iterate over the first nine pages

In [58]:
count = 0
total_pages = int(input('Enter the number of pages to iterate over. Below 5 works best.'))

while count != total_pages: 
    print('Page :', count+1)
    count = count + 1

    # Get source
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')  

    # Max_iterator = len(SOUP_JOBS)
    #iterator = 0
    
    # Make a SOUP iterable list
    SOUP_JOBS = soup.find_all(class_= 'jobsearch-SerpJobCard unifiedRow row result clickcard')
        
    # Make a SELENIUM iterable list
    SELENIUM_JOBS = driver.find_elements_by_xpath("//div[@class = 'jobsearch-SerpJobCard unifiedRow row result clickcard']")
        
    # Run loop to store things in CSV
    for i in range(3) :
        
        # Make variables to store info and use later
        Job_Title = SOUP_JOBS[i].find(attrs={'data-tn-element' : 'jobTitle'})['title']
        Company_Name = SOUP_JOBS[i].find(class_ = 'company').text
        Location = SOUP_JOBS[i].find(class_ = 'location accessible-contrast-color-location').text
        Salary = 'NEGOTIABLE'
        if SOUP_JOBS[i].find(class_ = 'salaryText') != None:
            Salary = SOUP_JOBS[i].find(class_ = 'salaryText').text
            Salary
        Job_Summary = ''           
        
        # Focus on element, click it
        try:
            SELENIUM_JOBS[i].click()
        except ElementClickInterceptedException:
            driver.find_element_by_partial_link_text('No, thanks').click()
            print('Pop up closed while clicking card!!')
            SELENIUM_JOBS[i].click()
        
        # Only if new page has opened
        num_tabs = driver.window_handles
        if len(num_tabs) == 2:
            
            # Switch to new tab
            driver.switch_to_window(num_tabs[1])
            
            # Find and store Job_Summary
            Job_Summary = driver.find_element_by_xpath("//div[@class='jobsearch-ViewJobLayout-jobDisplay icl-Grid-col icl-u-xs-span12 icl-u-lg-span7']").text

            # Move on in life
            driver.close()
            try:
                driver.switch_to_window(num_tabs[0])
            except ElementClickInterceptedException:
                driver.find_element_by_partial_link_text('No, thanks').click()
                print('Pop up closed while switching back!!')
                driver.switch_to_window(num_tabs[0])
                    
            
            
            if len(driver.window_handles) != 1:
                print('OHHHHH!!! OHHHHHHHHHH!!! ERROR!!!!!!')
                print('DROP EVERYTHING AND CALL HELP!!! THIS IS THE BIGGEST ERROR EVER!!!')
                print('Please, PLEASE, call Jaspreet!')
                break
                    
        else:
            print('OHHHHH!!! OHHHHHHHHHH!!! ERROR!!!!!!')
            print('Please, PLEASE, call Jaspreet!')
            break
        
        
        
        #creating an entry to add to CSV file
        csv_dict = [{'Job_Title':Job_Title, 'Company_Name':Company_Name, 'Location':Location, 'Salary':Salary, 'Job_Summary':Job_Summary}]
        temp_df_entry = pd.DataFrame(csv_dict)
        
        #pushing entry into CSV file
        df = df.append(temp_df_entry, ignore_index = True)
        df.to_csv('Chalo.csv')
    
    try:
        next_page = driver.find_element_by_partial_link_text('Next »')
        next_page.click()
    except ElementClickInterceptedException:
        driver.find_element_by_partial_link_text('No, thanks').click()
        print('Pop Up Closed!!')
        next_page = driver.find_element_by_partial_link_text('Next »')
        next_page.click()
    #No Next------------------------------

Enter the number of pages to iterate over. Below 5 works best.3
Page : 1


/home/jaspreet/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: DeprecationWarning: use driver.switch_to.window instead
/home/jaspreet/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: use driver.switch_to.window instead


Page : 2
Pop up closed while clicking card!!
Page : 3


In [44]:
driver.find_elements_by_xpath("//div[@class = 'location accessible-contrast-color-location']")

[]

In [45]:
driver.current_url

'https://www.indeed.co.in/jobs?q=Sales&l=Delhi&start=80'